In [ ]:
!pip install bs4

In [16]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = "https://ksp.karnataka.gov.in"

# Step 1: Get the initial HTML content
response = requests.get(base_url + "/firsearch/en")
soup = BeautifulSoup(response.content, 'html.parser')

# Step 2: Extract district options
district_select = soup.find('select', {'id': 'district_id'})
districts = []

for option in district_select.find_all('option'):
    value = option.get('value')
    text = option.text.strip()
    if value and value != "0":
        districts.append({'district_id': value, 'district_name': text})

districts_df = pd.DataFrame(districts)
print("Districts:")
print(districts_df)

# Step 3: Simulate AJAX request for police stations
def fetch_police_stations(district_id):
    url = f"{base_url}/myform/ajax/{district_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return []

all_police_stations = []
for district in districts:
    district_id = district['district_id']
    police_stations = fetch_police_stations(district_id)
    for station in police_stations:
        all_police_stations.append({
            'district_id': district_id,
            'district_name': district['district_name'],
            'station_id': station['station_id'],
            'station_name': station['unit_name']
        })

police_stations_df = pd.DataFrame(all_police_stations)
print("\nPolice Stations:")
print(police_stations_df)

# Step 4: Save results to CSV
districts_df.to_csv('districts.csv', index=False)
police_stations_df.to_csv('police_stations.csv', index=False)

print("\nData has been saved to CSV files: districts.csv and police_stations.csv.")


Districts:
   district_id            district_name
0            1                 Bagalkot
1            2                  Ballari
2            3            Belagavi City
3            4            Belagavi Dist
4            5           Bengaluru City
5            6           Bengaluru Dist
6            7                    Bidar
7            8           Chamarajanagar
8            9           Chickballapura
9           10           Chikkamagaluru
10          11              Chitradurga
11          12                      CID
12          13  Coastal Security Police
13          14         Dakshina Kannada
14          15               Davanagere
15          16                  Dharwad
16          17                    Gadag
17          18                   Hassan
18          19                   Haveri
19          20    Hubballi Dharwad City
20          21            ISD Bengaluru
21          22                    K.G.F
22          23               Kalaburagi
23          24          Kalab

In [17]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <meta content="no-cache" http-equiv="cache-control"/>
  <meta content="0" http-equiv="expires"/>
  <meta content="no-cache" http-equiv="pragma"/>
  <meta content="gadagpolice" name="description">
   <meta content="gadagpolice,gadag police, gadag city police" name="keywords">
    <meta content="gadagpolice" name="author">
     <link href="https://dri.karnataka.gov.in/frontend/jquery.ime/css/jquery.ime.css" rel="stylesheet"/>
     <meta content="notranslate" name="google"/>
     <style>
      .dp{display:block !important;}
        @import "https://ceg.karnataka.gov.in/frontend/opt1/css/global.css";
.modal-footer{display:block !important;}
.icon-bar2{z-index:9999999999999999999 !Important;top:85% !important;}
  .slideoverlay{    position: absolute;
    right: -19px;
    top: 46%;
    z-index: 99;
    width: 14%;}
	.slideoverlayleftg20 img{width:125

In [22]:
import requests
from bs4 import BeautifulSoup
import os

# Base URL
base_url = "https://ksp.karnataka.gov.in"

# Station IDs to test (replace with actual list)
stations_url = f"{base_url}/myform/ajax/5?unit_name=5"
stations = requests.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]

# Headers for the request
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": base_url,
    "Referer": f"{base_url}/firsearch/en",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Function to fetch the captcha value
def get_captcha():
    response = requests.get(f"{base_url}/firsearch/en", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    captcha_value = soup.find("input", {"name": "random_captcha"})["value"]
    print(captcha_value)
    return captcha_value

# Loop through police stations and FIR numbers
for ps_id in ps_ids[:5]:  # Limit to first 5 stations for testing
    for fir_num in range(1, 6):  # Testing for FIR numbers 0001 to 0005
        # Fetch the captcha value dynamically
        random_captcha = get_captcha()

        # Prepare the POST data
        data = {
            "district_id": "5",  # Replace with actual district ID
            "ps_id": ps_id,
            "fir_num": f"{fir_num:04}",
            "year": "2023",
            "knen": "en",
            "random_captcha": random_captcha,
            "captcha": random_captcha,  # Submit the same value as entered
        }

        # Make the POST request
        response = requests.post(f"{base_url}/fir_search.php", headers=headers, data=data)

        # Check response for the PDF link
        soup = BeautifulSoup(response.content, "html.parser")
        print(soup.prettify())
        pdf_link_tag = soup.find("a", href=True)

        if not pdf_link_tag:
            print(f"No PDF link found for station {ps_id} and FIR number {data['fir_num']}")
            continue

        # Extract and download the PDF
        pdf_url = base_url + pdf_link_tag["href"]
        full_path = os.path.join(".", str(ps_id))
        os.makedirs(full_path, exist_ok=True)

        with open(f"{full_path}/fir_{data['fir_num']}.pdf", "wb") as pdf_file:
            pdf_file.write(requests.get(pdf_url).content)
        print(f"Downloaded FIR {data['fir_num']} for station {ps_id}")


QHZGX
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0001
UKWJU
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0002
RRTKD
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0003
XOLEM
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0004
GCLHO
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0005


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

# Base URL
base_url = "https://ksp.karnataka.gov.in"

# Headers for the request
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": base_url,
    "Referer": f"{base_url}/firsearch/en",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Initialize session
session = requests.Session()

# Function to fetch the captcha and CSRF token
def get_captcha_and_csrf():
    response = session.get(f"{base_url}/firsearch/en", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract captcha value
    captcha_value = soup.find("input", {"name": "random_captcha"})["value"]

    # Extract CSRF token
    csrf_token = soup.find("input", {"name": "csrf_token"})["value"]

    print(f"Captcha: {captcha_value}, CSRF Token: {csrf_token}")
    return captcha_value, csrf_token

# Loop through police stations and FIR numbers
stations_url = f"{base_url}/myform/ajax/5?unit_name=5" 
stations = session.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]

for ps_id in ps_ids[:5]:  # Limit to first 5 stations for testing
    for fir_num in range(10, 16):  # Testing for FIR numbers 0001 to 0005
        # Fetch captcha and CSRF token dynamically
        random_captcha, csrf_token = get_captcha_and_csrf()

        # Prepare the POST data
        data = {
            "district_id": "1",  # Replace with actual district ID
            "ps_id": 1245,
            "fir_num": f"{fir_num:04}",
            "year": "2024",
            "knen": "en",
            "random_captcha": random_captcha,
            "captcha": random_captcha,  # Submit the same value as entered
            "csrf_token": csrf_token,  # Include the CSRF token
        }

        # Make the POST request
        response = session.post(f"{base_url}/fir_search.php", headers=headers, data=data)

        # Check response for the PDF link
        soup = BeautifulSoup(response.content, "html.parser")
        print(soup.prettify())

        # Find the 'View FIR Copy' link using the button's class or other identifiers
        pdf_link_tag = soup.find("a", class_="btn btn-primary btn-lg mt-3 d-block mx-auto")

        if pdf_link_tag and pdf_link_tag.get("href"):
            # Extract the full PDF URL
            pdf_url = pdf_link_tag["href"]
            print(f"PDF link found: {pdf_url}")
            
            # Download the PDF
            full_path = os.path.join(".", str(ps_id))
            os.makedirs(full_path, exist_ok=True)
            
            with open(f"{full_path}/fir_{data['fir_num']}.pdf", "wb") as pdf_file:
                pdf_file.write(requests.get(pdf_url).content)
            
            print(f"Downloaded FIR {data['fir_num']} for station {ps_id}")
        else:
            print(f"No PDF link found for station {ps_id} and FIR number {data['fir_num']}")


Captcha: BLJWN, CSRF Token: 3d3be4a3ec7be3af365ac25825d2d84ffae0ab0a7309a322e73f8caff7927143
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0010
Captcha: WNCXV, CSRF Token: 64fe6843b2df6155ab15cf1f82d699d4f0906cf96fd4c48c1c28857f11f7f937
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0011
Captcha: YRFGS, CSRF Token: 8b9dd295da26b3a5e24b0bfa1b301ca136f9aa91c2c4f956a9a8319e6b759fcf
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css" rel="stylesheet"/>
<div class="container">
</div>

No PDF link found for station 1382 and FIR number 0012
Captcha: YCJYV, CSRF Token: 521b291f7fcb9ee3d8ce6c2ca1082bebb65045a5b1f036af39502fe32088fb68
<link href="https://ksp.karnataka.gov.in/frontend/opt1/css/bootstrap.min.css

KeyboardInterrupt: 

In [31]:
import requests
from bs4 import BeautifulSoup
import time
import os

# Base URL
base_url = "https://ksp.karnataka.gov.in"

# Headers for the request
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": base_url,
    "Referer": f"{base_url}/firsearch/en",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Initialize session
session = requests.Session()

# Function to fetch the captcha and CSRF token
def get_captcha_and_csrf():
    response = session.get(f"{base_url}/firsearch/en", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract captcha value
    captcha_value = soup.find("input", {"name": "random_captcha"})["value"]

    # Extract CSRF token
    csrf_token = soup.find("input", {"name": "csrf_token"})["value"]

    print(f"Captcha: {captcha_value}, CSRF Token: {csrf_token}")
    return captcha_value, csrf_token

# Loop through police stations and FIR numbers
stations_url = f"{base_url}/myform/ajax/5?unit_name=5"
stations = session.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]

for ps_id in ps_ids[:5]:  # Limit to first 5 stations for testing
    for fir_num in range(1, 6):  # Testing for FIR numbers 0001 to 0005
        # Fetch captcha and CSRF token dynamically
        random_captcha, csrf_token = get_captcha_and_csrf()

        # Prepare the POST data
        data = {
            "district_id": "5",  # Replace with actual district ID
            "ps_id": ps_id,
            "fir_num": f"{fir_num:04}",
            "year": "2023",
            "knen": "en",
            "random_captcha": random_captcha,
            "captcha": random_captcha,  # Submit the same value as entered
            "csrf_token": csrf_token,  # Include the CSRF token
        }

        # Debugging: Print POST data and send the request
        print(f"Data Sent: {data}")

        # Make the POST request
        response = session.post(f"{base_url}/fir_search.php", headers=headers, data=data, allow_redirects=True)

        # Debugging: Check status and headers
        print(f"Status Code: {response.status_code}")
        print(f"Response Headers: {response.headers}")

        # Check if the response contains the expected content
        print(f"Response Content: {response.text[:1000]}")  # Print first 1000 characters of the response

        # Add delay to avoid rate limiting
        time.sleep(2)

        # Parse response for the PDF link
        soup = BeautifulSoup(response.content, "html.parser")
        pdf_link_tag = soup.find("a", href=True)

        if not pdf_link_tag:
            print(f"No PDF link found for station {ps_id} and FIR number {data['fir_num']}")
            continue

        # Extract and download the PDF
        pdf_url = base_url + pdf_link_tag["href"]
        full_path = os.path.join(".", str(ps_id))
        os.makedirs(full_path, exist_ok=True)

        with open(f"{full_path}/fir_{data['fir_num']}.pdf", "wb") as pdf_file:
            pdf_file.write(requests.get(pdf_url).content)
        print(f"Downloaded FIR {data['fir_num']} for station {ps_id}")


Captcha: JICHX, CSRF Token: 8ac7076362e6d543ef59a0da8a8b066c807eda5d35df538f86e79a3c1295c9c0
Data Sent: {'district_id': '5', 'ps_id': 1382, 'fir_num': '0001', 'year': '2023', 'knen': 'en', 'random_captcha': 'JICHX', 'captcha': 'JICHX', 'csrf_token': '8ac7076362e6d543ef59a0da8a8b066c807eda5d35df538f86e79a3c1295c9c0'}
Status Code: 200
Response Headers: {'Date': 'Mon, 30 Dec 2024 15:57:06 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/7.3.20', 'Strict-Transport-Security': 'max-age=63072000; includeSubdomains; preload', 'X-Content-Type-Options': 'nosniff', 'X-Xss-Protection': '1; mode=block', 'Content-Security-Policy': "frame-ancestors 'self' https://*.karnataka.gov.in;", 'X-Frame-Options': 'SAMEORIGIN', 'X-Permitted-Cross-Domain-Policies': 'none', 'Referrer-Policy': 'same-origin', 'Access-Control-Allow-Origin': '*', 'Keep-Alive': 'timeout=5, max=98', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked', 'Content-Type': 'text/html; charset=UTF-8'}
Response Content: <link href="https:

KeyboardInterrupt: 

In [35]:
import requests
from bs4 import BeautifulSoup
import os
import time

# Base URL
base_url = "https://ksp.karnataka.gov.in"

# Headers for the request
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": base_url,
    "Referer": f"{base_url}/firsearch/en",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Initialize session
session = requests.Session()

# Function to fetch the captcha and CSRF token
def get_captcha_and_csrf():
    response = session.get(f"{base_url}/firsearch/en", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract captcha value
    captcha_value = soup.find("input", {"name": "random_captcha"})["value"]

    # Extract CSRF token
    csrf_token = soup.find("input", {"name": "csrf_token"})["value"]

    print(f"Captcha: {captcha_value}, CSRF Token: {csrf_token}")
    return captcha_value, csrf_token

# Loop through police stations and FIR numbers
stations_url = f"{base_url}/myform/ajax/5?unit_name=5"
stations = session.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]

for ps_id in ps_ids[:5]:  # Limit to first 5 stations for testing
    for fir_num in range(1, 6):  # Testing for FIR numbers 0001 to 0005
        # Fetch captcha and CSRF token dynamically
        random_captcha, csrf_token = get_captcha_and_csrf()

        # Prepare the POST data
        data = {
            "district_id": "5",  # Replace with actual district ID
            "ps_id": ps_id,
            "fir_num": f"{fir_num:04}",
            "year": "2023",
            "knen": "en",
            "random_captcha": random_captcha,
            "captcha": random_captcha,  # Submit the same value as entered
            "csrf_token": csrf_token,  # Include the CSRF token
        }

        # Make the POST request
        response = session.post(f"{base_url}/fir_search.php", headers=headers, data=data)

        # Check response for the PDF link
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Print the raw HTML for debugging
        # print(soup.prettify())

        # Look for the 'View FIR Copy' button/link
        pdf_link_tag = soup.find("a", class_="btn btn-primary btn-lg mt-3 d-block mx-auto", href=True)

        if pdf_link_tag:
            # Extract and download the PDF
            pdf_url = pdf_link_tag["href"]
            print(f"Found PDF link: {pdf_url}")

            # Create the folder for saving the PDF
            full_path = os.path.join(".", str(ps_id))
            os.makedirs(full_path, exist_ok=True)

            # Download and save the PDF
            with open(f"{full_path}/fir_{data['fir_num']}.pdf", "wb") as pdf_file:
                pdf_file.write(requests.get(pdf_url).content)
            print(f"Downloaded FIR {data['fir_num']} for station {ps_id}")
        else:
            print(f"No PDF link found for station {ps_id} and FIR number {data['fir_num']}")
        
        # Add delay to avoid rate limiting
        time.sleep(2)

Captcha: GLHFO, CSRF Token: 26711419e1e45550cd2a7c58ba5e833873d1c55636bc7112209e7b64f44914f4
No PDF link found for station 1382 and FIR number 0001
Captcha: GZJKS, CSRF Token: fd73f00f1719c2dbaf00fab467495ee5fdf19f8906afd39931a69e0bab57d1f8
No PDF link found for station 1382 and FIR number 0002
Captcha: MAWDF, CSRF Token: 6a3d6561b7d461f76294a232b8b313722517bc736bf64f6fce9aa9929434f9c3
No PDF link found for station 1382 and FIR number 0003
Captcha: JDVQN, CSRF Token: 2bc5e976ae9f4a9246c946a1f02f96d1d22dc2ed71584d5017559418f5911b9c
No PDF link found for station 1382 and FIR number 0004
Captcha: SXZYE, CSRF Token: 3a3c220692b4e5976c14a50cb9079ca0606036c298e5bc7b6e66c2b86c82b522
No PDF link found for station 1382 and FIR number 0005
Captcha: TCJPX, CSRF Token: 451452fc569af6d249282d790035ade49fac52a02f7dff2dccdf9826ddf03049
No PDF link found for station 1762 and FIR number 0001
Captcha: NUJPN, CSRF Token: f0890fcbdb794da09c28dcd089b27ffaf404fcea7ee52615257668c0a0843fe1
No PDF link found f

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

base_url = "https://ksp.karnataka.gov.in"
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": base_url,
    "Referer": f"{base_url}/firsearch/en",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

session = requests.Session()

def get_captcha_and_csrf():
    response = session.get(f"{base_url}/firsearch/en", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    captcha_value = soup.find("input", {"name": "random_captcha"})["value"]
    csrf_token = soup.find("input", {"name": "csrf_token"})["value"]
    
    return captcha_value, csrf_token

def search_fir(ps_id, fir_num, year):
    random_captcha, csrf_token = get_captcha_and_csrf()
    
    # Updated form data structure
    data = {
        "district_id": "5",
        "ps_id": ps_id,
        "fir_num": f"{fir_num:04d}",
        "year": str(year),
        "knen": "en",
        "random_captcha": random_captcha,
        "captcha": random_captcha,
        "csrf_token": csrf_token,
        "submit": "Submit"  # Added submit button value
    }
    
    # Make initial form submission
    response = session.post(
        f"{base_url}/firsearch/en",  # Changed endpoint
        headers=headers,
        data=data,
        allow_redirects=True
    )
    
    # Follow redirect if successful
    if response.history:
        response = session.get(response.url)
    
    return response

# Main execution code remains the same
stations_url = f"{base_url}/myform/ajax/5?unit_name=5"
stations = session.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]

for ps_id in ps_ids[:5]:
    for fir_num in range(1, 6):
        try:
            response = search_fir(ps_id, fir_num, 2024)
            
            if response.status_code == 500:
                print(f"Server error for station {ps_id} and FIR {fir_num}")
                continue
                
            soup = BeautifulSoup(response.content, "html.parser")
            print(soup.prettify())
            pdf_link = soup.find("a", class_="btn btn-primary btn-lg mt-3 d-block mx-auto", href=True)
            
            if pdf_link:
                pdf_url = pdf_link["href"]
                full_path = os.path.join(".", str(ps_id))
                os.makedirs(full_path, exist_ok=True)
                
                pdf_response = session.get(pdf_url)
                if pdf_response.status_code == 200:
                    with open(f"{full_path}/fir_{fir_num:04d}.pdf", "wb") as pdf_file:
                        pdf_file.write(pdf_response.content)
                    print(f"Downloaded FIR {fir_num:04d} for station {ps_id}")
            else:
                print(f"No PDF link found for station {ps_id} and FIR number {fir_num:04d}")
                
        except Exception as e:
            print(f"Error processing station {ps_id}, FIR {fir_num}: {str(e)}")
        
        time.sleep(2)

<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="noindex,nofollow" name="robots"/>
  <style>
   body { background-color: #fff; color: #222; font: 16px/1.5 -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; margin: 0; }
                .container { margin: 30px; max-width: 600px; }
                h1 { color: #dc3545; font-size: 24px; }
  </style>
 </head>
 <body>
  <div class="container">
   <h1>
    Whoops, looks like something went wrong.
   </h1>
  </div>
 </body>
</html>

No PDF link found for station 1382 and FIR number 0001
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="noindex,nofollow" name="robots"/>
  <style>
   body { background-color: #fff; color: #222; font: 16px/1.5 -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; margin: 0; }
                .container { margin: 30px; max-width: 600px; }
                h1 { color: #dc3545; font-size: 24

KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

# Base URL
base_url = "https://ksp.karnataka.gov.in"

# Headers for the request
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": base_url,
    "Referer": f"{base_url}/firsearch/en",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# Initialize session
session = requests.Session()

# Function to fetch the captcha and CSRF token
def get_captcha_and_csrf():
    response = session.get(f"{base_url}/firsearch/en", headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract captcha value
    captcha_value = soup.find("input", {"name": "random_captcha"})["value"]

    # Extract CSRF token
    csrf_token = soup.find("input", {"name": "csrf_token"})["value"]

    print(f"Captcha: {captcha_value}, CSRF Token: {csrf_token}")
    return captcha_value, csrf_token

# Fixed Police Station ID
ps_id = 1245  # Example of a fixed PS ID, replace with the actual one you want

# Loop through FIR numbers for the fixed PS ID
for fir_num in range(10, 16):  # Testing for FIR numbers 0001 to 0005
    # Fetch captcha and CSRF token dynamically
    random_captcha, csrf_token = get_captcha_and_csrf()

    # Prepare the POST data
    data = {
        "district_id": "1",  # Replace with actual district ID
        "ps_id": ps_id,
        "fir_num": f"{fir_num:04}",
        "year": "2024",
        "knen": "en",
        "random_captcha": random_captcha,
        "captcha": random_captcha,  # Submit the same value as entered
        "csrf_token": csrf_token,  # Include the CSRF token
    }

    # Make the POST request
    response = session.post(f"{base_url}/fir_search.php", headers=headers, data=data)

    # Check response for the PDF link
    soup = BeautifulSoup(response.content, "html.parser")
    print(soup.prettify())

    # Find the 'View FIR Copy' link using the button's class or other identifiers
    pdf_link_tag = soup.find("a", class_="btn btn-primary btn-lg mt-3 d-block mx-auto")

    if pdf_link_tag and pdf_link_tag.get("href"):
        # Extract the full PDF URL
        pdf_url = pdf_link_tag["href"]
        print(f"PDF link found: {pdf_url}")

        # Download the PDF
        full_path = os.path.join(".", str(ps_id))
        os.makedirs(full_path, exist_ok=True)

        with open(f"{full_path}/fir_{data['fir_num']}.pdf", "wb") as pdf_file:
            pdf_file.write(requests.get(pdf_url).content)

        print(f"Downloaded FIR {data['fir_num']} for station {ps_id}")
    else:
        print(f"No PDF link found for station {ps_id} and FIR number {data['fir_num']}")


Captcha: OBIRN, CSRF Token: 7049dbfb91bf5499055c6e05b4a0756d7bf750a19572292bffac80fc388beee3
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="noindex,nofollow" name="robots"/>
  <style>
   body { background-color: #fff; color: #222; font: 16px/1.5 -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, "Helvetica Neue", Arial, sans-serif; margin: 0; }
                .container { margin: 30px; max-width: 600px; }
                h1 { color: #dc3545; font-size: 24px; }
  </style>
 </head>
 <body>
  <div class="container">
   <h1>
    Whoops, looks like something went wrong.
   </h1>
  </div>
 </body>
</html>

No PDF link found for station 1245 and FIR number 0010
Captcha: YASHE, CSRF Token: 85b0a07bb0ce7b6e2c3df9f3fe3012f49e14ce52933dde4e1e58f633eef0fa12
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="noindex,nofollow" name="robots"/>
  <style>
   body { background-color: #fff; color: #222; font: 16px/1.5 -apple-system, BlinkMacSystemFon

In [ ]:
import requests, json, os
from bs4 import BeautifulSoup

url = "https://ksp.karnataka.gov.in/"
stations_url = "https://ksp.karnataka.gov.in/myform/ajax/5?unit_name=5"
stations = requests.get(stations_url).json()
ps_ids = [obj["station_id"] for obj in stations]
#   {
#     "id": 279,
#     "unit_name": "Yelahanka PS",
#     "circle_name": "Yelahanka Sub-Division",
#     "sub_division_name": "DCP North East Division",
#     "phone": "080-22942536",
#     "email": "email@ksp.gov.in",
#     "station_address": "YELAHANKA PS, YELAHANKA, BANGALORE-64",
#     "district_row_id": 5,
#     "station_id": 1355,
#     "latitude": 13.09474,
#     "longitude": 77.59442,
#     "created_at": "2021-07-19 16:24:32",
#     "updated_at": "2021-07-19 16:24:32"
#   }

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Origin": "https://ksp.karnataka.gov.in",
    "Referer": "https://ksp.karnataka.gov.in/firsearch/en",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

data = {
    "district_id": "5",
    "ps_id": 1382,
    "fir_num": "0027",
    "year": "2023",
    "knen": "en",
    "random_captcha": "LOL",
    "captcha": "LOL",
}

# Remove list slicing to iterate through all Stations
for id in ps_ids[:5]:
    # Increase range to 0000 to iterate till fir num 9999
    for num in range(1, 6):
        data["ps_id"] = id
        data["fir_num"] = f"{num:04}"
        response = requests.post(
            "https://ksp.karnataka.gov.in/fir_search_new_s.php", headers=headers, data=data
        )

        if "FIR Not Found!" in response.text:
            print(f"FIR not found for station {id} and fir number {data['fir_num']}")
            continue

        soup = BeautifulSoup(response.content, "html.parser")
        a_href = soup.a["href"]
        pdf_url = url + a_href
        full_path = os.path.join(".", str(id))
        if not os.path.exists(full_path):
            os.makedirs(full_path)

        with open(f"{id}/fir_{data['fir_num']}.pdf", "wb") as f:
            f.write(requests.get(pdf_url).content)